## DM lesson4
Dans ce dataset: https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv, chaque ligne correspond à un produit alimentaire mis en vente par un utilisateur.

Objectif: cleaner le dataset.

- On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur.
- La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.
Voic une liste d'APIs qui peut vous être utile : https://github.com/public-apis/public-apis (mais vous pouvez en utiliser d'autres si vous le voulez).

In [1]:
import pandas as pd
import requests

In [2]:
df = pd.read_csv('products.csv',sep=';')

In [3]:
df.head()

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


In [4]:
df.groupby('infos').count().head()

,username,ip_address,product,price
infos,,,,
Contains,1,1,1,1
Contains fish,2,2,2,2
Contains gluten,1,1,1,1
Contains gluten and egg,1,1,1,1
Contains gluten and sugar,4,4,4,4


In [5]:
# Clean price column
df['currency'] = df['price'].str.split(" ").str[1]
df['price'] = df['price'].str.split(" ").str[0]
pd.to_numeric(df['price'],errors='raise')
df.head()

,username,ip_address,product,price,infos,currency
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,NaN
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,NaN
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,NaN
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,NaN
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",NaN


In [6]:
# Clean infos
df['infos'] = df['infos'].str.lower()
df['infos'] = df['infos'].str.replace('may','')
df['infos'] = df['infos'].str.replace('contains','')
df['infos'] = df['infos'].str.replace('contain','')
df['infos'] = df['infos'].str.replace('ingredients','')
df['infos'] = df['infos'].str.replace(':','')
df['infos'] = df['infos'].str.replace('and ','')
df['infos'] = df['infos'].str.replace(', ',' ')
df['infos'] = df['infos'].str.replace('  ',' ')
df2 = df.join(df.infos.str.get_dummies(' '))

In [7]:
#del df2['infos']
df2.head(20)

,username,ip_address,product,price,infos,currency,egg,fish,gluten,milk,mustard,peanut,soja,sugar
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,sugar,NaN,0,0,0,0,0,0,0,1
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,peanut fish,NaN,0,1,0,0,0,1,0,0
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,mustard fish,NaN,0,1,0,0,1,0,0,0
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,gluten,NaN,0,0,1,0,0,0,0,0
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,sugar egg fish,NaN,1,1,0,0,0,0,0,1
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,sugar milk,MGA,0,0,0,1,0,0,0,1
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,sugar,NaN,0,0,0,0,0,0,0,1
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,sugar,NaN,0,0,0,0,0,0,0,1
8,epridham8,187.129.113.105,Dried Figs,88.05,sugar milk fish,NaN,0,1,0,1,0,0,0,1
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,peanut sugar milk fish,NaN,0,1,0,1,0,1,0,1


In [8]:
def checkCountry (IP):
    if requests.get('https://extreme-ip-lookup.com/json/'+ IP).json()['status'] == 'fail':
        return None
    else:
        return requests.get('https://extreme-ip-lookup.com/json/' + IP).json()['countryCode']

In [9]:
df2['country'] = df2.ip_address.apply(checkCountry)

In [10]:
def getCurrency(country):
    if country == None:
        return None
    if country == '':
        return None
    else:
        return requests.get('https://restcountries.eu/rest/v2/alpha/'+country).json()['currencies'][0]['code']

In [11]:
df2.currency = df2.country.apply(getCurrency)

In [12]:
def getExchangeRate(devise):
    if devise == None:
        return 1 
    if 'rates' not in requests.get('https://api.ratesapi.io/api/latest?base=USD;symbols='+devise).json():
        return 1
    else:
        return float(requests.get('https://api.ratesapi.io/api/latest?base=USD;symbols='+devise).json()['rates'][devise])

In [14]:
df2['price in $'] = round(df2.price.astype(float) / df2.currency.apply(getExchangeRate).astype(float),2)

In [15]:
df2.sort_values('price in $',ascending=False).head(20)

,username,ip_address,product,price,infos,currency,egg,fish,gluten,milk,mustard,peanut,soja,sugar,country,price in $
48,tnusche1c,109.144.241.78,Tomatoes - Grape,852.58,peanut sugar fish,GBP,0,1,0,0,0,1,0,1,GB,1099.23
17,ksumptonh,150.214.20.140,Bagel - Ched Chs Presliced,957.7,peanut sugar egg,EUR,1,0,0,0,0,1,0,1,ES,1067.26
114,aatack36,158.197.55.255,Tamarind Paste,946.48,gluten sugar fish,EUR,0,1,1,0,0,0,0,1,SK,1054.76
64,hreidshaw1s,104.137.59.250,"Rice Pilaf, Dry,package",992.68,soja sugar egg,USD,1,0,0,0,0,0,1,1,US,992.68
106,nlampkin2y,49.215.30.109,Tomato - Plum With Basil,990.81,peanut mustard sugar egg,TWD,1,0,0,0,1,1,0,1,TW,990.81
49,ychuter1d,136.251.73.188,"Coconut - Shredded, Unsweet",985.06,peanut mustard sugar milk,USD,0,0,0,1,1,1,0,1,US,985.06
69,psemechik1x,38.360.77.106,Curry Paste - Madras,976.51,mustard sugar milk fish,None,0,1,0,1,1,0,0,1,None,976.51
10,cjagielaa,75.254.207.163,Cinnamon Rolls,966.34,peanut sugar egg fish,USD,1,1,0,0,0,1,0,1,US,966.34
118,zembleton3a,134.10.200.254,Wine - Pinot Noir Latour,944.51,soja sugar milk,USD,0,0,0,1,0,0,1,1,US,944.51
120,cgorusso3c,16.157.249.370,"Pasta - Shells, Medium, Dry",940.51,gluten sugar,None,0,0,1,0,0,0,0,1,None,940.51
